In [ ]:
# Based on MNIST CNN from Keras' examples: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py (MIT License)

from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np

"""
https://paperswithcode.com/dataset/kuzushiji-49
https://github.com/rois-codh/kmnist

['http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-imgs.npz',
'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-labels.npz',
'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-imgs.npz',
'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-labels.npz'],
"""

In [24]:
batch_size = 128
num_classes = 49
epochs = 250

# input image dimensions
img_rows, img_cols = 28, 28

def load(f):
    return np.load(f)['arr_0']

# Load the data
x_train = load('k49-train-imgs.npz')
x_test = load('k49-test-imgs.npz')
y_train = load('k49-train-labels.npz')
y_test = load('k49-test-labels.npz')

In [25]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('{} train samples, {} test samples'.format(len(x_train), len(x_test)))

232365 train samples, 38547 test samples


In [26]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [27]:
# Model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/250
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 163s 89ms/step - accuracy: 0.0306 - loss: 3.8790 - val_accuracy: 0.0823 - val_loss: 3.8419
Epoch 2/250
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 176s 97ms/step - accuracy: 0.0648 - loss: 3.8220 - val_accuracy: 0.1346 - val_loss: 3.7623
Epoch 3/250
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 176s 97ms/step - accuracy: 0.0989 - loss: 3.7273 - val_accuracy: 0.1864 - val_loss: 3.6266
Epoch 4/250
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 175s 96ms/step - accuracy: 0.1447 - loss: 3.5729 - val_accuracy: 0.2493 - val_loss: 3.4329
Epoch 5/250
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 174s 96ms/step - accuracy: 0.1948 - loss: 3.3681 - val_accuracy: 0.3009 - val_loss: 3.2186
Epoch 6/250
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 174s 96ms/step - accuracy: 0.2374 - loss: 3.1717 - val_accuracy: 0.3298 - val_loss: 3.0302
Epoch 7/250
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 174s 96ms/step - accuracy: 0.2732 - loss: 3.0041 - val_accuracy: 0.3528 - val_loss: 2.8797
Epoch 8/250
1816/1816 ━━━━━━━━━━━━━━━━━━━━ 176s 97ms/step - ac

In [30]:
# Metrics
train_score = model.evaluate(x_train, y_train, verbose=0)
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

Train loss: 0.5431498289108276
Train accuracy: 0.8624534606933594
Test loss: 0.9898993968963623
Test accuracy: 0.7455573678016663


In [31]:
# Realizar predicciones en todo el conjunto de prueba
predictions = model.predict(x_test)

# Obtener las clases predichas
predicted_classes = np.argmax(predictions, axis=1)

# Obtener las clases verdaderas
true_classes = np.argmax(y_test, axis=1)

# Contar cuántas predicciones son correctas
correct_predictions = np.sum(predicted_classes == true_classes)

# Calcular la precisión
accuracy = correct_predictions / len(y_test)

print(f"Total de imágenes en el conjunto de prueba: {len(y_test)}")
print(f"Predicciones correctas: {correct_predictions}")
print(f"Precisión en el conjunto de prueba: {accuracy:.2%}")


1205/1205 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step
Total de imágenes en el conjunto de prueba: 38547
Predicciones correctas: 28739
Precisión en el conjunto de prueba: 74.56%


In [33]:
model.save('kanji_model_250_epochs.hdf5')